# An Introduction to LlamaIndex Query Pipelines

## Overview
LlamaIndex provides a declarative query API that allows you to chain together different modules in order to orchestrate simple-to-advanced workflows over your data.

This is centered around our `QueryPipeline` abstraction. Load in a variety of modules (from LLMs to prompts to retrievers to other pipelines), connect them all together into a sequential chain or DAG, and run it end2end.

**NOTE**: You can orchestrate all these workflows without the declarative pipeline abstraction (by using the modules imperatively and writing your own functions). So what are the advantages of `QueryPipeline`? 

- Express common workflows with fewer lines of code/boilerplate
- Greater readability
- Greater parity / better integration points with common low-code / no-code solutions (e.g. LangFlow)
- [In the future] A declarative interface allows easy serializability of pipeline components, providing portability of pipelines/easier deployment to different systems.

## Cookbook

In this cookbook we give you an introduction to our `QueryPipeline` interface and show you some basic workflows you can tackle.

- Chain together prompt and LLM
- Chain together query rewriting (prompt + LLM) with retrieval
- Chain together a full RAG query pipeline (query rewriting, retrieval, reranking, response synthesis)
- Setting up a custom query component

## Setup

Here we setup some data + indexes (from PG's essay) that we'll be using in the rest of the cookbook.

In [ ]:
%pip install llama-index-embeddings-voyageai
%pip install llama-index-postprocessor-voyageai-rerank
%pip install llama-index-llms-openai

In [ ]:
# setup Arize Phoenix for logging/observability
import phoenix as px

px.launch_app()
import llama_index.core

llama_index.core.set_global_handler("arize_phoenix")

🌍 To view the Phoenix app in your browser, visit http://127.0.0.1:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.voyageai import VoyageEmbedding
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = VoyageEmbedding(model_name="voyage-large-2")

In [ ]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader("../data/paul_graham")

In [ ]:
docs = reader.load_data()

In [ ]:
import os
from llama_index.core import (
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)

if not os.path.exists("storage"):
    index = VectorStoreIndex.from_documents(docs)
    # save index to disk
    index.set_index_id("vector_index")
    index.storage_context.persist("./storage")
else:
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="storage")
    # load index
    index = load_index_from_storage(storage_context, index_id="vector_index")

## 1. Chain Together Prompt and LLM 

In this section we show a super simple workflow of chaining together a prompt with LLM.

We simply define `chain` on initialization. This is a special case of a query pipeline where the components are purely sequential, and we automatically convert outputs into the right format for the next inputs.

In [ ]:
from llama_index.core.query_pipeline import QueryPipeline
from llama_index.core import PromptTemplate

# try chaining basic prompts
prompt_str = "Please generate related movies to {movie_name}"
prompt_tmpl = PromptTemplate(prompt_str)
llm = OpenAI(model="gpt-3.5-turbo")

p = QueryPipeline(chain=[prompt_tmpl, llm], verbose=True)

In [ ]:
output = p.run(movie_name="The Departed")

> Running module 1331b23f-d409-4b21-b9f7-a45b3c3c7031 with input: 
movie_name: The Departed

> Running module bfe6afb2-25ed-429f-a180-1c7581b1d5fa with input: 
messages: Please generate related movies to The Departed



In [ ]:
print(str(output))

assistant: 1. Infernal Affairs (2002) - The original Hong Kong film that inspired The Departed
2. The Town (2010) - A crime thriller directed by Ben Affleck
3. Mystic River (2003) - A crime drama directed by Clint Eastwood
4. Goodfellas (1990) - A classic crime film directed by Martin Scorsese
5. The Irishman (2019) - Another crime film directed by Martin Scorsese, starring Robert De Niro and Al Pacino
6. The Godfather (1972) - A classic crime film directed by Francis Ford Coppola
7. Heat (1995) - A crime thriller directed by Michael Mann, starring Al Pacino and Robert De Niro
8. The Departed (2006) - A crime thriller directed by Martin Scorsese, starring Leonardo DiCaprio and Matt Damon
9. Casino (1995) - A crime film directed by Martin Scorsese, starring Robert De Niro and Joe Pesci
10. American Gangster (2007) - A crime film directed by Ridley Scott, starring Denzel Washington and Russell Crowe.


### Try Output Parsing

Let's parse the outputs into a structured Pydantic object.

In [ ]:
from typing import List
from pydantic import BaseModel, Field
from llama_index.core.output_parsers import PydanticOutputParser


class Movie(BaseModel):
    """Object representing a single movie."""

    name: str = Field(..., description="Name of the movie.")
    year: int = Field(..., description="Year of the movie.")


class Movies(BaseModel):
    """Object representing a list of movies."""

    movies: List[Movie] = Field(..., description="List of movies.")


llm = OpenAI(model="gpt-3.5-turbo")
output_parser = PydanticOutputParser(Movies)
json_prompt_str = """\
Please generate related movies to {movie_name}. Output with the following JSON format: 
"""
json_prompt_str = output_parser.format(json_prompt_str)

In [ ]:
# add JSON spec to prompt template
json_prompt_tmpl = PromptTemplate(json_prompt_str)

p = QueryPipeline(chain=[json_prompt_tmpl, llm, output_parser], verbose=True)
output = p.run(movie_name="Toy Story")

> Running module e16375ef-6d2a-4297-a2de-26376a7af0b9 with input: 
movie_name: Toy Story

> Running module 4acbc86f-36de-45e5-b9fb-1f79e7149f3a with input: 
messages: Please generate related movies to Toy Story. Output with the following JSON format: 



Here's a JSON schema to follow:
{"$defs": {"Movie": {"description": "Object representing a single movie.", "prop...

> Running module 4e3c0f02-e277-4eab-9131-9ed0d04471d6 with input:
input: assistant: {
"movies": [
{
"name": "Finding Nemo",
"year": 2003
},
{
"name": "Monsters, Inc.",
"year": 2001
},
{
"name": "The Incredibles",
"y...



In [ ]:
output

Movies(movies=[Movie(name='Finding Nemo', year=2003), Movie(name='Monsters, Inc.', year=2001), Movie(name='The Incredibles', year=2004)])